Create Network Diagram using NetworkX

In [5]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import altair as alt

data =  pd.read_csv("Clinvar/harmful_mutations.csv")
data.head()

,Gene Name,Mutation ID Number,Mutation Position,Mutation Value,Clinical Significance,Clinical Significance Grouped,Disease Implicated
0,9636,183381,1:1014143,C->T,Pathogenic,Pathogenic,Immunodeficiency 38 with basal ganglia calcifi...
1,9636,161455,1:1014316,C->CG,Pathogenic,Pathogenic,Immunodeficiency 38 with basal ganglia calcifi...
2,9636,161454,1:1014359,G->T,Pathogenic,Pathogenic,Immunodeficiency 38 with basal ganglia calcifi...
3,375790,243036,1:1022225,G->A,Pathogenic,Pathogenic,Congenital myasthenic syndrome
4,375790,243037,1:1022313,A->T,Pathogenic,Pathogenic,Congenital myasthenic syndrome


In [7]:
# Filter top 20 mutation values
top_mutations = data['Mutation Value'].value_counts().head(20).index
top_df = data[data['Mutation Value'].isin(top_mutations)]

# Create network graph
G = nx.from_pandas_edgelist(top_df, 'Gene Name', 'Disease Implicated', ['Clinical Significance Grouped'])

# Convert the graph to a format Altair can use
nodes, edges = [], []
for node in G.nodes():
    nodes.append({'id': node})
for edge in G.edges(data=True):
    edges.append({'source': edge[0], 'target': edge[1], 'Clinical Significance Grouped': edge[2]['Clinical Significance Grouped']})

nodes_df = pd.DataFrame(nodes)
edges_df = pd.DataFrame(edges)

# Create Altair chart for nodes
nodes_chart = alt.Chart(nodes_df).mark_circle(size=400).encode(
    x=alt.X('id:N', axis=None),
    y=alt.Y('id:N', axis=None),
    tooltip=['id']
).properties(
    width=800,
    height=800
)

# Create Altair chart for edges
edges_chart = alt.Chart(edges_df).mark_line().encode(
    x='source:N',
    y='target:N',
    color='Clinical Significance Grouped:N'
)

# Combine nodes and edges
final_chart = nodes_chart + edges_chart
final_chart.configure_mark(
    opacity=0.8,
    color='lightblue'
).interactive()

final_chart.display()

alt.LayerChart(...)